In [ ]:
from jupyter_imports import *

In [ ]:
from riskpnl.post_trade import batch_summarize_exec_logs

start = datetime(2022,6,23)#,tzinfo=timezone.utc)
log_path = os.path.join(os.sep, 'tmp','prod','tradeexecutor')
log_file = os.path.join(os.sep,'tmp','prod','tradeexecutor','all_by_symbol.csv')
batch_summarize_exec_logs(log_path,start=start,add_history_context=False)
original_weights = pd.read_csv(log_file).sort_values(by='log_time')
original_weights['log_time'] = original_weights['log_time'].apply(parser.parse)

In [ ]:
weights = original_weights[original_weights['log_time']>start.replace(tzinfo=timezone.utc)]

weights['total_bps'] = weights['slippage_bps'] + weights['fee']/weights['filledUSD']*10000
weights['fee_USD'] = weights['fee']
weights['slippage_USD'] = weights['slippage_bps']*weights['filledUSD']/10000
weights['total_USD'] = weights['total_bps']*weights['filledUSD']/10000

### cumulative slippage and fees

In [ ]:
weights

In [ ]:
trackrecord = weights[weights['index']=='average'].pivot_table(index='log_time',values=['fee_USD','slippage_USD','total_USD'],aggfunc=np.mean).cumsum()

In [ ]:
trackrecord.iplot()

### slippage distribution

In [ ]:
(weights.loc[weights['index']=='average','total_bps']).hist(bins=50,figsize=(20,10))